## **빈도 분석 TF_IDF**

In [92]:
from konlpy.tag import Okt
from math import log
import pandas as pd
import numpy as np
import os
import re
okt = Okt()

In [93]:
cd "C:\Users\82105\Desktop\프로젝트 7조\인스타그램 크롤링 코드"

C:\Users\82105\Desktop\프로젝트 7조\인스타그램 크롤링 코드


In [94]:
df = pd.read_csv("tokenized.csv", engine='python', encoding='cp949')

#### 사전작업

In [160]:
# 레스토랑 목록
unique = df['restaurant'].unique()

In [161]:
# bag of words 코드
word_index = {}
bow = []
token = okt.morphs("")

for voca in token:
    if voca not in word_index.keys():
        word_index[voca] = len(word_index)
        bow.insert(len(word_index)-1, 1)
    else:
        index = word_index.get(voca)
        bow[index] = bow[index] +1

print(word_index)
print(bow)

{}
[]


In [162]:
# groupby 식당별 행 묶기 (globals로 동적 변수 생성)
for res in unique:
    call = df['review'].groupby(df['restaurant']).get_group(res)
    review = ""
    for sentence in call:
        review = review + " " + sentence
    globals()['{}'.format(res)] = review

In [163]:
reviews = [
    겐로쿠우동, 고기공방, 고물상, 깔리, 나누미떡볶이, 낙원테산도, 대통령삼겹살, 대포찜닭, 뎁짜이, 도토리편백집, 독일주택, 돈돈정, 마로화적, 머노까머나, 메밀향그집, 멘야산다이메, 미도인, 미분당, 바오쯔, 버거파크, 불난집, 브라운에비뉴, 삼겹살전문점대통령, 서양집, 소친친, 솔나무길된장예술, 오이지, 유자유김치떡볶이, 육미안, 정돈, 쪼리375, 칸다소바, 파스타마켓, 페르시안궁전, 포크랜드, 피터팬스테이크, 하이디라오, 현초밥, 혜화도담,
    혜화칼국수, 호호식당, 혼가츠
]

In [ ]:
review_all = ""

for i in reviews:
    review_all = review_all + " " + i

print(review_all)

#### 토큰화

In [101]:
# 토큰화 (stem = True: '맛있는' -> '맛있다' 기본형 출력)
tokenized_sentence = okt.morphs(review_all, stem=True) 

In [ ]:
# stopwords에 포함되지 않는 형태소를 tokenized 리스트에 단 한 번씩만 담기도록
tokenized = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다', '나']

for word in tokenized_sentence:
    if word not in stopwords:
        if word not in tokenized:
            tokenized.append(word)

print(tokenized)

#### 빈도분석

In [151]:
# 빈도분석하고자 하는 키워드
vocab = ["데이트", "남자친구", "여자친구", "남자", "여자", "애인", "남친", "여친", "친구", "부모", "부모님", "연극", "혼자", "혼밥", "효도", "웨이팅", "기다리다", "분위기", "가성비", "비싸다", "저렴하다", "싸다", "대학생", "맛있다", "학생", "키즈", "아이", "생일", "기념일", "청결하다", "더럽다", "불편하다", "좋다", "별로다", "어머니", "아버지", "할머니", "할아버지", "환갑", "생신", "모임", "존맛탱", "존맛", "기념", "어린이날", "어버이날", "고급", "가족", "딸", "아들", "회식"] 

In [142]:
# tf, idf, tfidf 함수
n = len(reviews)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in reviews:
        df += t in doc
    return log(n/(df + 1))

def tfidf(t, d):
    return tf(t, d) * idf(t)

In [152]:
# tf
result = []

for i in range(n):
    result.append([])
    d = reviews[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))

tf_selected = pd.DataFrame(result, columns = vocab)
tf_selected

,데이트,남자친구,여자친구,남자,여자,애인,남친,여친,친구,부모,...,존맛탱,존맛,기념,어린이날,어버이날,고급,가족,딸,아들,회식
0,1,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,59,0,0,2,1,1,2,0,7,3,...,3,8,2,1,0,2,1,1,0,0
2,3,0,0,0,0,0,0,0,9,0,...,0,6,0,0,0,2,0,0,0,0
3,48,1,0,8,10,0,2,1,32,0,...,8,15,0,0,0,0,2,4,3,0
4,41,0,0,5,6,0,0,0,23,2,...,4,20,1,1,0,2,7,14,6,1
5,18,0,1,1,2,0,0,0,4,0,...,7,16,0,0,0,0,1,1,0,0
6,1,0,0,1,0,0,0,0,2,0,...,3,4,0,0,0,0,1,3,0,3
7,26,0,0,0,1,0,0,1,6,0,...,6,15,2,2,0,0,0,1,1,1
8,57,0,0,4,2,0,0,0,18,0,...,8,21,1,0,0,11,2,5,3,1
9,4,2,0,4,0,0,0,0,6,1,...,0,2,1,0,0,0,0,0,1,0


In [167]:
# tfidf
result = []
for i in range(n):
    result.append([])
    d = reviews[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t, d))

tfidf_selected = pd.DataFrame(result, columns = vocab)
tfidf_selected

,데이트,남자친구,여자친구,남자,여자,애인,남친,여친,친구,부모,...,존맛탱,존맛,기념,어린이날,어버이날,고급,가족,딸,아들,회식
0,0.048790,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.148216,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.878620,0.000000,0.000000,0.543867,0.336472,1.94591,2.870169,0.000000,0.518756,3.518161,...,0.546965,0.390321,1.037588,1.540445,0.000000,1.694596,0.370374,0.370374,0.000000,0.000000
2,0.146370,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.666972,0.000000,...,0.000000,0.292741,0.000000,0.000000,0.000000,1.694596,0.000000,0.000000,0.000000,0.000000
3,2.341928,0.741937,0.000000,2.175470,3.364722,0.00000,2.870169,1.791759,2.371455,0.000000,...,1.458572,0.731852,0.000000,0.000000,0.000000,0.000000,0.740748,1.481495,1.438719,0.000000
4,2.000397,0.000000,0.000000,1.359669,2.018833,0.00000,0.000000,0.000000,1.704483,2.345441,...,0.729286,0.975803,0.518794,1.540445,0.000000,1.694596,2.592617,5.185233,2.877438,0.965081
5,0.878223,0.000000,1.658228,0.271934,0.672944,0.00000,0.000000,0.000000,0.296432,0.000000,...,1.276251,0.780643,0.000000,0.000000,0.000000,0.000000,0.370374,0.370374,0.000000,0.000000
6,0.048790,0.000000,0.000000,0.271934,0.000000,0.00000,0.000000,0.000000,0.148216,0.000000,...,0.546965,0.195161,0.000000,0.000000,0.000000,0.000000,0.370374,1.111121,0.000000,2.895243
7,1.268544,0.000000,0.000000,0.000000,0.336472,0.00000,0.000000,1.791759,0.444648,0.000000,...,1.093929,0.731852,1.037588,3.080890,0.000000,0.000000,0.000000,0.370374,0.479573,0.965081
8,2.781039,0.000000,0.000000,1.087735,0.672944,0.00000,0.000000,0.000000,1.333943,0.000000,...,1.458572,1.024593,0.518794,0.000000,0.000000,9.320276,0.740748,1.851869,1.438719,0.965081
9,0.195161,1.483875,0.000000,1.087735,0.000000,0.00000,0.000000,0.000000,0.444648,1.172720,...,0.000000,0.097580,0.518794,0.000000,0.000000,0.000000,0.000000,0.000000,0.479573,0.000000


#### 각 식당별 빈도 비율 구하기

In [168]:
ratio_selected = tf_selected
a = ratio_selected.sum()

In [157]:
# 비율 계산
for i in range(len(ratio_selected.columns)):
    col = ratio_selected.columns[i]
    for j in range(len(ratio_selected)):
        ratio = round((ratio_selected[col].iloc[j] / a[i]),7)
        ratio_selected[col].iloc[j] = ratio


C:\Users\82105\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\82105\AppData\Local\Temp/ipykernel_1284/2795464776.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  ratio = round((ratio_selected[col].iloc[j] / a[i]),7)
C:\Users\82105\AppData\Local\Temp/ipykernel_1284/2795464776.py:4: RuntimeWarning: invalid value encountered in double_scalars
  ratio = round((ratio_selected[col].iloc[j] / a[i]),7)


In [148]:
ratio_selected

,데이트,남자친구,여자친구,남자,여자,애인,남친,여친,친구,부모,...,모임,존맛탱,존맛,기념,어린이날,어버이날,고급,가족,딸,아들
0,0.000631,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.004320,0.00,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.037248,0.000000,0.000000,0.017241,0.010989,0.166667,0.1250,0.000000,0.015119,0.12,...,0.011364,0.015152,0.017204,0.028169,0.1,0.000000,0.034483,0.011628,0.006369,0.000000
2,0.001894,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.019438,0.00,...,0.000000,0.000000,0.012903,0.000000,0.0,0.000000,0.034483,0.000000,0.000000,0.000000
3,0.030303,0.027778,0.000000,0.068965,0.109890,0.000000,0.1250,0.111111,0.069114,0.00,...,0.034091,0.040404,0.032258,0.000000,0.0,0.000000,0.000000,0.023256,0.025478,0.032258
4,0.025884,0.000000,0.000000,0.043103,0.065934,0.000000,0.0000,0.000000,0.049676,0.08,...,0.022727,0.020202,0.043011,0.014084,0.1,0.000000,0.034483,0.081395,0.089172,0.064516
5,0.011364,0.000000,0.066667,0.008621,0.021978,0.000000,0.0000,0.000000,0.008639,0.00,...,0.000000,0.035354,0.034409,0.000000,0.0,0.000000,0.000000,0.011628,0.006369,0.000000
6,0.000631,0.000000,0.000000,0.008621,0.000000,0.000000,0.0000,0.000000,0.004320,0.00,...,0.022727,0.015152,0.008602,0.000000,0.0,0.000000,0.000000,0.011628,0.019108,0.000000
7,0.016414,0.000000,0.000000,0.000000,0.010989,0.000000,0.0000,0.111111,0.012959,0.00,...,0.011364,0.030303,0.032258,0.028169,0.2,0.000000,0.000000,0.000000,0.006369,0.010753
8,0.035985,0.000000,0.000000,0.034483,0.021978,0.000000,0.0000,0.000000,0.038877,0.00,...,0.045455,0.040404,0.045161,0.014084,0.0,0.000000,0.189655,0.023256,0.031847,0.032258
9,0.002525,0.055556,0.000000,0.034483,0.000000,0.000000,0.0000,0.000000,0.012959,0.04,...,0.034091,0.000000,0.004301,0.014084,0.0,0.000000,0.000000,0.000000,0.000000,0.010753


In [43]:
# 인덱스 추가
tf_selected.insert(0, "name", ['겐로쿠우동', '고기공방', '고물상', '깔리', '나누미떡볶이', '낙원테산도', '대통령삼겹살', '대포찜닭', '뎁짜이', '도토리편백집', '독일주택', '돈돈정', '마로화적', '머노까머나', '메밀향그집', '멘야산다이메', '미도인', '미분당', '바오쯔', '버거파크', '불난집', '브라운에비뉴', '삼겹살전문점대통령', '서양집', '소친친', '솔나무길된장예술', '오이지', '유자유김치떡볶이', '육미안', '정돈', '쪼리375', '칸다소바', '파스타마켓', '페르시안궁전', '포크랜드', '피터팬스테이크', '하이디라오', '현초밥', '혜화도담', '혜화칼국수', '호호식당', '혼가츠'])

tfidf_selected.insert(0, "name", ['겐로쿠우동', '고기공방', '고물상', '깔리', '나누미떡볶이', '낙원테산도', '대통령삼겹살', '대포찜닭', '뎁짜이', '도토리편백집', '독일주택', '돈돈정', '마로화적', '머노까머나', '메밀향그집', '멘야산다이메', '미도인', '미분당', '바오쯔', '버거파크', '불난집', '브라운에비뉴', '삼겹살전문점대통령', '서양집', '소친친', '솔나무길된장예술', '오이지', '유자유김치떡볶이', '육미안', '정돈', '쪼리375', '칸다소바', '파스타마켓', '페르시안궁전', '포크랜드', '피터팬스테이크', '하이디라오', '현초밥', '혜화도담', '혜화칼국수', '호호식당', '혼가츠'])

ratio_selected.insert(0, "name", ['겐로쿠우동', '고기공방', '고물상', '깔리', '나누미떡볶이', '낙원테산도', '대통령삼겹살', '대포찜닭', '뎁짜이', '도토리편백집', '독일주택', '돈돈정', '마로화적', '머노까머나', '메밀향그집', '멘야산다이메', '미도인', '미분당', '바오쯔', '버거파크', '불난집', '브라운에비뉴', '삼겹살전문점대통령', '서양집', '소친친', '솔나무길된장예술', '오이지', '유자유김치떡볶이', '육미안', '정돈', '쪼리375', '칸다소바', '파스타마켓', '페르시안궁전', '포크랜드', '피터팬스테이크', '하이디라오', '현초밥', '혜화도담', '혜화칼국수', '호호식당', '혼가츠'])

In [44]:
tf_selected.to_csv("tf_selected.csv")
tfidf_selected.to_csv("tfidf_selected.csv")
ratio_selected.to_csv("keyword_frequency_ratio.csv")